# Семинар 5. Групповая обработка и слияние наборов данных в Pandas

На этом семинаре мы поработаем с данными о продажах глобального интернет-магазина, реализующего офисные товары и технику. Цель занятия - научиться использовать групповую обработку в Pandas и соединять таблицы данных.

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Загрузим данные:

In [2]:
orders = pd.read_csv('../datasets/global_superstore/orders.zip', encoding='utf-8')
orders.head()

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,postal_code,city,...,product_id,category,sub_category,product_name,sales,quantity,discount,profit,shipping_cost,order_priority
0,40098,CA-2014-AB10015140-41954,2014-11-11,2014-11-13,First Class,AB-100151402,Aaron Bergman,Consumer,73120.0,Oklahoma City,...,TEC-PH-5816,Technology,Phones,Samsung Convoy 3,221.980,2,0.0,62.1544,40.77,High
1,26341,IN-2014-JR162107-41675,2014-02-05,2014-02-07,Second Class,JR-162107,Justin Ritter,Corporate,NaN,Wollongong,...,FUR-CH-5379,Furniture,Chairs,"Novimex Executive Leather Armchair, Black",3709.395,9,0.1,-288.7650,923.63,Critical
2,25330,IN-2014-CR127307-41929,2014-10-17,2014-10-18,First Class,CR-127307,Craig Reiter,Consumer,NaN,Brisbane,...,TEC-PH-5356,Technology,Phones,"Nokia Smart Phone, with Caller ID",5175.171,9,0.1,919.9710,915.49,Medium
3,13524,ES-2014-KM1637548-41667,2014-01-28,2014-01-30,First Class,KM-1637548,Katherine Murray,Home Office,NaN,Berlin,...,TEC-PH-5267,Technology,Phones,"Motorola Smart Phone, Cordless",2892.510,5,0.1,-96.5400,910.16,Medium
4,47221,SG-2014-RH9495111-41948,2014-11-05,2014-11-06,Same Day,RH-9495111,Rick Hansen,Consumer,NaN,Dakar,...,TEC-CO-6011,Technology,Copiers,"Sharp Wireless Fax, High-Speed",2832.960,8,0.0,311.5200,903.04,Critical


In [3]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51290 entries, 0 to 51289
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   row_id          51290 non-null  int64  
 1   order_id        51290 non-null  object 
 2   order_date      51290 non-null  object 
 3   ship_date       51290 non-null  object 
 4   ship_mode       51290 non-null  object 
 5   customer_id     51290 non-null  object 
 6   customer_name   51290 non-null  object 
 7   segment         51290 non-null  object 
 8   postal_code     9994 non-null   float64
 9   city            51290 non-null  object 
 10  state           51290 non-null  object 
 11  country         51290 non-null  object 
 12  region          51290 non-null  object 
 13  market          51290 non-null  object 
 14  product_id      51290 non-null  object 
 15  category        51290 non-null  object 
 16  sub_category    51290 non-null  object 
 17  product_name    51290 non-null 

При загрузке со стандартными параметрами появилось несколько проблем:

- Столбец  почтового индекса (`postal_code`) распознано как числовой. Однако в некоторых странах почтовый индекс может содержать буквы и другие символы.

- Даты загрузились как текст

Добавим спецификацию форматов данных в команду для загрузки:

In [4]:
orders = pd.read_csv('../datasets/global_superstore/orders.zip', encoding='utf-8',
                    parse_dates=['order_date', 'ship_date'],
                    dtype={'postal_code' : str})
orders.head()

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,postal_code,city,...,product_id,category,sub_category,product_name,sales,quantity,discount,profit,shipping_cost,order_priority
0,40098,CA-2014-AB10015140-41954,2014-11-11,2014-11-13,First Class,AB-100151402,Aaron Bergman,Consumer,73120,Oklahoma City,...,TEC-PH-5816,Technology,Phones,Samsung Convoy 3,221.980,2,0.0,62.1544,40.77,High
1,26341,IN-2014-JR162107-41675,2014-02-05,2014-02-07,Second Class,JR-162107,Justin Ritter,Corporate,NaN,Wollongong,...,FUR-CH-5379,Furniture,Chairs,"Novimex Executive Leather Armchair, Black",3709.395,9,0.1,-288.7650,923.63,Critical
2,25330,IN-2014-CR127307-41929,2014-10-17,2014-10-18,First Class,CR-127307,Craig Reiter,Consumer,NaN,Brisbane,...,TEC-PH-5356,Technology,Phones,"Nokia Smart Phone, with Caller ID",5175.171,9,0.1,919.9710,915.49,Medium
3,13524,ES-2014-KM1637548-41667,2014-01-28,2014-01-30,First Class,KM-1637548,Katherine Murray,Home Office,NaN,Berlin,...,TEC-PH-5267,Technology,Phones,"Motorola Smart Phone, Cordless",2892.510,5,0.1,-96.5400,910.16,Medium
4,47221,SG-2014-RH9495111-41948,2014-11-05,2014-11-06,Same Day,RH-9495111,Rick Hansen,Consumer,NaN,Dakar,...,TEC-CO-6011,Technology,Copiers,"Sharp Wireless Fax, High-Speed",2832.960,8,0.0,311.5200,903.04,Critical


In [5]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51290 entries, 0 to 51289
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   row_id          51290 non-null  int64         
 1   order_id        51290 non-null  object        
 2   order_date      51290 non-null  datetime64[ns]
 3   ship_date       51290 non-null  datetime64[ns]
 4   ship_mode       51290 non-null  object        
 5   customer_id     51290 non-null  object        
 6   customer_name   51290 non-null  object        
 7   segment         51290 non-null  object        
 8   postal_code     9994 non-null   object        
 9   city            51290 non-null  object        
 10  state           51290 non-null  object        
 11  country         51290 non-null  object        
 12  region          51290 non-null  object        
 13  market          51290 non-null  object        
 14  product_id      51290 non-null  object        
 15  ca

Теперь данные загрузились правильно. Даты стали датами, а `postal_code` теперь - текстовое поле. Количество пропущенных значений не изменилось, в данных действительно во многих случаях индекс не указан.

При загрузке из текстовых файлов может возникать множество проблем из-за ошибок в данных или несоответствия форматов. При необходимости обратитесь к ресурсам:  

- справка Pandas https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html
- Раздел 6.1 в книге Уэса Маккинни Python и анализ данных (см. ресурсы в LMS)

**Словарь данных:**

 - `row_id` - код строки заказа (уникальный)  
 - `order_id` - код заказа
 - `order_date` - дата размещения заказа
 - `ship_date` - дата отгрузки заказа (пусто, если еще не отгружен)
 - `ship_mode` - класс отправки
 - `customer_id` - код клиента
 - `customer_name` - имя клиента
 - `segment` - рыночный сегмент клиента
 - `postal_code` - почтовый индекс получателя
 - `city` - город получателя
 - `state` - штат получателя
 - `country` - страна получателя
 - `region` - регион получателя
 - `market` - рынок, к которому относится получатель
 - `product_id` - код товара
 - `category` - товарная категория
 - `sub_category` - товарная подкатегория
 - `product_name` - наименование товара
 - `sales` - выручка
 - `quantity` - количество единиц товара
 - `discount` - скидка
 - `profit` - валовая прибыль 
 - `shipping_cost` - затраты на доставку
 - `order_priority` - приоритет заказа
 

Данные представляют собой строки заказов, выполненных компанией. Каждая строка имеет уникальный номер `row_id` и содержит данные о том, кто, что и когда заказал, когда эта позиция была отправлена, адрес получателя. Также количество единиц, выручку, прибыль и затраты на доставку по строке. 

В одном заказе может быть несколько позиций. В этом случае в таблице будет несколько строк с одинаковым номером заказа:

In [6]:
orders.query('order_id == "AE-2015-GH4665138-42351"')

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,postal_code,city,...,product_id,category,sub_category,product_name,sales,quantity,discount,profit,shipping_cost,order_priority
27486,43926,AE-2015-GH4665138-42351,2015-12-13,2015-12-19,Standard Class,GH-4665138,Greg Hansen,Consumer,NaN,Ras al Khaymah,...,TEC-PH-5253,Technology,Phones,"Motorola Headset, VoIP",95.796,4,0.7,-156.564,6.73,Medium
29298,43924,AE-2015-GH4665138-42351,2015-12-13,2015-12-19,Standard Class,GH-4665138,Greg Hansen,Consumer,NaN,Ras al Khaymah,...,OFF-ST-4078,Office Supplies,Storage,"Eldon Lockers, Blue",59.373,1,0.7,-118.767,5.83,Medium
33521,43922,AE-2015-GH4665138-42351,2015-12-13,2015-12-19,Standard Class,GH-4665138,Greg Hansen,Consumer,NaN,Ras al Khaymah,...,OFF-AR-3463,Office Supplies,Art,"BIC Pencil Sharpener, Water Color",77.256,8,0.7,-82.584,4.08,Medium
40033,43925,AE-2015-GH4665138-42351,2015-12-13,2015-12-19,Standard Class,GH-4665138,Greg Hansen,Consumer,NaN,Ras al Khaymah,...,OFF-ST-5693,Office Supplies,Storage,"Rogers File Cart, Single Width",42.489,1,0.7,-62.331,2.24,Medium
49469,43923,AE-2015-GH4665138-42351,2015-12-13,2015-12-19,Standard Class,GH-4665138,Greg Hansen,Consumer,NaN,Ras al Khaymah,...,OFF-BI-3246,Office Supplies,Binders,"Avery Binder Covers, Clear",3.159,1,0.7,-4.971,1.25,Medium
49470,43927,AE-2015-GH4665138-42351,2015-12-13,2015-12-19,Standard Class,GH-4665138,Greg Hansen,Consumer,NaN,Ras al Khaymah,...,OFF-BI-3249,Office Supplies,Binders,"Avery Binder Covers, Recycled",3.429,1,0.7,-3.891,1.25,Medium


## Задание 1

Во всех заданиях необходимо округлить результат до разумного количества десятичных знаков. Крупные суммы выводите в подходящих единицах (тысячи или млн долларов)

### 1.1. Сколько составляет оборот по каждой товарной категории? 
(выведите ответ в млн долларов с округлением до 1 десятичного знака).

### 1.2. Сколько товаров представлено в каждой товарной категории? 
(попробуйте реализовать двумя способами: только с помощью группировки/определения размера группы, и через агрегирующую функцию `nunique()` - число уникальных значений.

### 1.3. Составьте рейтинг стран на основе прибыли от продаж.

### 1.4. Выведите 3 самых популярных (по количеству заказов) товара на каждом рынке.

### 1.5. Сравните рынки по характеристикам заказов на них:
- Средний и медианный чек (сумма заказа)
- Среднее и медианное количество позиций в заказе
- Средняя и медианная прибыль по заказу
- Средняя и медианная стоимость доставки по заказу
- Среднее и медианное отношение стоимости доставки заказа к его сумме

Обратите внимание, что анализировать нужно именно характеристики заказов. В таблице содержатся _строки_ заказов, сначала на их основе нужно посчитать характеристики заказов.

### 1.6. Рейтинг по относительной доле затрат на доставку
Рассчитайте для каждой строки (позиции) заказа долю затрат на доставку к выручке по этой строке. Для каждого рынка составьте рейтинг из 5 самых дорогих по стоимости доставке позиций. Чтобы их найти, посчитайте для каждой строки отличие доли затрат на доставку этой позиции к средней доле затрат на доставку для этого региона, товарной категории и класса доставки.

**Подсказка:** для подсчета отличий по группам используйте `transform()`.

### 1.7. Сравните динамику продаж по годам различных товарных подкатегорий категории `Technology`.

Попробуйте два способа: через `pivot_table()` и через `unstack()`.

**Подсказка**: компоненты даты - например, год, месяц можно получить на основе столбца с датами с помощью атрибутов `dt.year`, `dt.month`, `dt.day`:

In [7]:
orders.head(5)['order_date']

0   2014-11-11
1   2014-02-05
2   2014-10-17
3   2014-01-28
4   2014-11-05
Name: order_date, dtype: datetime64[ns]

In [8]:
orders.head(5)['order_date'].dt.year

0    2014
1    2014
2    2014
3    2014
4    2014
Name: order_date, dtype: int64

In [9]:
orders.head(5)['order_date'].dt.month

0    11
1     2
2    10
3     1
4    11
Name: order_date, dtype: int64

In [10]:
orders.head(5)['order_date'].dt.day

0    11
1     5
2    17
3    28
4     5
Name: order_date, dtype: int64

### 1.8. Сравните динамику продаж по годам товаров брендов Apple и Samsung

### 1.9. Составьте рейтинг стран по прибыли.
В списке должны быть только страны, в которые отправлено более 500 заказов.

## Задание 2

Дополним данные сведениями о том, какие заказы были возвращены.

In [11]:
returns = pd.read_csv('../datasets/global_superstore/returns.csv')
returns.head()

,returned,order_id,region
0,True,CA-2012-SA20830140-41210,Central US
1,True,IN-2012-PB19210127-41259,Eastern Asia
2,True,CA-2012-SC20095140-41174,Central US
3,True,IN-2015-JH158207-42140,Oceania
4,True,IN-2014-LC168857-41747,Oceania


In [12]:
returns.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1079 entries, 0 to 1078
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   returned  1079 non-null   bool  
 1   order_id  1079 non-null   object
 2   region    1079 non-null   object
dtypes: bool(1), object(2)
memory usage: 18.0+ KB


В таблице `returns` содержатся записи о тех заказах, которые были возвращены - номер заказа, регион и статус возврата.

Довольно необычной особенностью этого набора данных является то, что номера заказов в нем не уникальны. Заказы с одинаковым номером могли быть зарегистрированы в разных регионах:

In [13]:
(orders[['order_id', 'region']]
 .groupby('order_id',)
 .filter(lambda grp: grp['region'].nunique() >1)
 .reset_index(drop=True))

,order_id,region
0,CA-2015-AG10330140-42361,Western US
1,CA-2015-AG10330140-42361,Central US
2,CA-2015-AG10330140-42361,Central US
3,CA-2015-AG10330140-42361,Central US
4,CA-2014-BW11200140-41940,Eastern US
5,CA-2014-BW11200140-41940,Southern US
6,CA-2014-BW11200140-41940,Eastern US
7,CA-2014-BW11200140-41940,Eastern US
8,CA-2014-BW11200140-41940,Eastern US
9,CA-2012-CS12355140-41135,Southern US


По этой причине соединять таблицы необходимо по двум ключам - `order_id` и `region`.

Попробуем соединить таблицы: 

In [14]:
orders_returns = orders.merge(returns, on=['order_id', 'region']) # лучше явно указать ключи

In [15]:
orders_returns.shape

(2202, 25)

Строк получилось очень мало. В исходном наборе было более 50 тысяч строк:

In [16]:
orders.shape

(51290, 24)

### 2.1. Почему пропали строки? 
Исправьте аргументы `merge()`, чтобы задать правильный тип соединения.

### 2.2. Составьте рейтинг регионов по убыванию доли возвращенных заказов.

### 2.3. Рейтинг менеджеров

В еще одной таблице - `people` - содержатся данные о менеджерах, которые руководят продажами в различных регионах:

In [17]:
people = pd.read_csv('../datasets/global_superstore/people.csv')
people.head()

,person,region
0,Marilène Rousseau,Caribbean
1,Andile Ihejirika,Central Africa
2,Nicodemo Bautista,Central America
3,Cansu Peynirci,Central Asia
4,Lon Bonher,Central US


Соедините таблицу `orders` с этими данными и выведите рейтинг менеджеров по прибыли.

### 2.4. Объединение таблиц
На основе таблицы `orders` определите три самые прибыльные товарные подкатегории на рынке США и сохраните их в таблице `best_usa`. Затем на основе этой же таблицы определите три самые прибыльные подкатегории на рынке Европы и сохраните их в таблице `best_europe`. Затем объедините данные из таблиц `best_usa` и `best_europe` в одной таблице.